In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from community_simulator.usertools import (MakeConsumerDynamics, MakeResourceDynamics,  MakeParams,
                                           MakeMatrices, MakeInitialState, a_default)
from community_simulator import Community
from community_simulator.visualization import PlotTraj, StackPlot
import pickle
from biom import Table, load_table
from scipy.spatial.distance import pdist, squareform
from skbio.stats.ordination import pcoa
%matplotlib inline

from torchdiffeq import odeint_adjoint as odeint
from torch_bnn import BNN

In [2]:
!ls

C.txt  neural-ode.ipynb  preprocess.ipynb  simple-sim.ipynb  torch_bnn.py
D.txt  N.txt		 R.txt		   split.ipynb


In [11]:
C = pd.read_table('C.txt', index_col=1, header=1)
D = pd.read_table('D.txt', index_col=1, header=1)
N = pd.read_table('N.txt', index_col=0, header=1)
R = pd.read_table('R.txt', index_col=0, header=1)

In [24]:
C = C.iloc[:, 1:]
D = D.iloc[:, 1:]
N = N.rename(columns={'Unnamed: 6': 'time'})
R = R.rename(columns={'Unnamed: 91': 'time'})

In [ ]:
# TODO: Build NN / BNN to draw function
# Figure out initial value
odeint(func, y0, t)